In [1]:
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import numpy as np

In [2]:
#Now we define a new dataframe called data from yfinance which will contain the stock price of apple on a specified date range.
# Define the stock ticker and date range
ticker = 'AMZN'
start_date = '2010-04-01'
end_date = '2024-11-23'

# Fetch the data using yfinance
data = yf.download(ticker, start=start_date, end=end_date)

# Display the DataFrame
data

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2010-04-01,6.790000,6.825500,6.559000,6.590500,6.590500,175716000
2010-04-05,6.642500,6.687000,6.539000,6.574500,6.574500,116330000
2010-04-06,6.561500,6.800000,6.559000,6.778000,6.778000,159006000
2010-04-07,6.798000,6.804000,6.693000,6.743500,6.743500,118908000
2010-04-08,6.735500,7.062500,6.735500,7.048000,7.048000,253782000
...,...,...,...,...,...,...
2024-11-18,204.149994,204.669998,200.949997,201.699997,201.699997,36512500
2024-11-19,199.330002,205.300003,198.779999,204.610001,204.610001,31197900
2024-11-20,202.979996,203.130005,199.449997,202.880005,202.880005,32769000


In [3]:
import numpy as np
data['VWAP'] = (data['Close'] * data['Volume']).cumsum() / data['Volume'].cumsum()

# Calculate the log difference for 'Open', 'Close', 'High', 'Low', 'Volume'
columns_to_calculate = ['Open', 'Close', 'High', 'Low', 'Volume', 'VWAP']
for column in columns_to_calculate:
    data[f'log_diff_{column}'] = np.log(data[column] / data[column].shift(1))

data.loc[:, 'Target'] = (data['Close'].shift(-1) > data['Close']).astype(int)


# List of columns to calculate logarithms for
columns_to_log = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'VWAP']

# Ensure the columns exist in the DataFrame before processing
existing_columns = [col for col in columns_to_log if col in data.columns]

# Add new columns for logarithms
for column in existing_columns:
    data[f'log_{column}'] = np.log(data[column])

In [4]:
horizons = [2,5,60,250,1000]
new_predictors = []
data['Log_Returns'] = np.log(data['Close'] / data['Close'].shift(1))
for horizon in horizons:
    rolling_averages = data.rolling(horizon).mean()
    
    ratio_column = f"Close_Ratio_{horizon}"
    data[ratio_column] = data["Close"] / rolling_averages["Close"]
    
    trend_column = f"Trend_{horizon}"
    data[trend_column] = data.shift(1).rolling(horizon).sum()["Target"]
    data[f'Volatility_{horizon}'] = data['Log_Returns'].rolling(window=horizon).std()

    new_predictors+= [ratio_column, trend_column]

# Ensure 'Close' contains the daily closing prices


# Calculate rolling volatility (e.g., 20-day rolling window)
# Remove rows with any null values
data = data.dropna()
data

,Open,High,Low,Close,Adj Close,Volume,VWAP,log_diff_Open,log_diff_Close,log_diff_High,...,Volatility_5,Close_Ratio_60,Trend_60,Volatility_60,Close_Ratio_250,Trend_250,Volatility_250,Close_Ratio_1000,Trend_1000,Volatility_1000
Date,,,,,,,,,,,,,,,,,,,,,
2014-03-24,18.004499,18.075001,17.430000,17.592501,17.592501,97470000,10.517801,-0.029848,-0.024620,-0.030887,...,0.013803,0.937302,28.0,0.021579,1.091307,132.0,0.017852,1.543695,510.0,0.020746
2014-03-25,17.701500,17.948500,17.441999,17.735500,17.735500,88914000,10.524604,-0.016972,0.008096,-0.007023,...,0.013068,0.947010,28.0,0.021531,1.098959,132.0,0.017822,1.554720,511.0,0.020747
2014-03-26,17.856501,17.879999,17.170000,17.170500,17.170500,82414000,10.530405,0.008718,-0.032376,-0.003824,...,0.015723,0.919077,28.0,0.021814,1.062936,131.0,0.017946,1.503821,510.0,0.020752
2014-03-27,17.157499,17.200001,16.544001,16.923500,16.923500,115328000,10.538204,-0.039932,-0.014490,-0.038773,...,0.015532,0.908080,28.0,0.021835,1.046650,131.0,0.017931,1.480868,510.0,0.020757
2014-03-28,17.002501,17.350000,16.804001,16.914499,16.914499,79736000,10.543578,-0.009075,-0.000532,0.008683,...,0.016680,0.910059,27.0,0.021733,1.045124,130.0,0.017927,1.478804,509.0,0.020712
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-18,204.149994,204.669998,200.949997,201.699997,201.699997,36512500,64.009242,-0.012704,-0.004502,-0.012961,...,0.025409,1.068257,33.0,0.018643,1.140440,131.0,0.017179,1.361395,511.0,0.022183
2024-11-19,199.330002,205.300003,198.779999,204.610001,204.610001,31197900,64.023812,-0.023893,0.014324,0.003073,...,0.026118,1.080892,34.0,0.018652,1.155308,132.0,0.017166,1.380619,511.0,0.022187
2024-11-20,202.979996,203.130005,199.449997,202.880005,202.880005,32769000,64.038924,0.018146,-0.008491,-0.010626,...,0.020649,1.068952,34.0,0.018588,1.144088,131.0,0.017139,1.368535,511.0,0.022187


In [5]:
features = ['Log_Returns','Close_Ratio_2', 'Trend_2', 'Close_Ratio_5', 'Trend_5','Close_Ratio_60', 'Trend_60', 'Volatility_60',
       'Close_Ratio_250', 'Trend_250', 'Volatility_250', 'Close_Ratio_1000',
       'Trend_1000', 'Volatility_1000', 'log_Open', 'log_High', 'log_Low',
       'log_Close', 'log_Adj Close', 'log_Volume', 'log_VWAP']
target = ['Target']

In [6]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import numpy as np

# Assuming `data` is your DataFrame
features = features
target = ['Target']

# Split data into training (80%) and testing (20%) sets
split_index = int(len(data) * 0.8)
train_data = data.iloc[:split_index]
test_data = data.iloc[split_index:]

# Separate features (X) and target (y)
X_train = train_data[features]
y_train = train_data[target].values.ravel()
X_test = test_data[features]
y_test = test_data[target].values.ravel()

# Initialize and train the Random Forest model
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)

# Predict probabilities for the test set
y_prob = rf_model.predict_proba(X_test)[:, 1]  # Probability for the positive class

# Custom threshold for classification
custom_threshold = 0.4  # Change this to your desired threshold
y_pred = (y_prob >= custom_threshold).astype(int)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

# Print evaluation results
print(f"Accuracy: {accuracy:.2f}")
print("\nConfusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(class_report)

# Optional: Display feature importances
feature_importances = pd.DataFrame({
    'Feature': features,
    'Importance': rf_model.feature_importances_
}).sort_values(by='Importance', ascending=False)

print("\nFeature Importances:")
print(feature_importances)


Accuracy: 0.52

Confusion Matrix:
[[ 99 161]
 [ 97 181]]

Classification Report:
              precision    recall  f1-score   support

           0       0.51      0.38      0.43       260
           1       0.53      0.65      0.58       278

    accuracy                           0.52       538
   macro avg       0.52      0.52      0.51       538
weighted avg       0.52      0.52      0.51       538


Feature Importances:
             Feature  Importance
19        log_Volume    0.069200
3      Close_Ratio_5    0.063940
5     Close_Ratio_60    0.063536
7      Volatility_60    0.059761
0        Log_Returns    0.058511
1      Close_Ratio_2    0.058079
11  Close_Ratio_1000    0.056262
10    Volatility_250    0.055298
8    Close_Ratio_250    0.055006
13   Volatility_1000    0.053141
20          log_VWAP    0.048923
14          log_Open    0.048770
15          log_High    0.048525
16           log_Low    0.046094
18     log_Adj Close    0.044960
17         log_Close    0.043489
12       

In [7]:
# Reset the index to make 'Date' a column
data = data.reset_index()

# Extract only the date part and convert to string format
data['Date'] = data['Date'].dt.date.astype(str)
data

,Date,Open,High,Low,Close,Adj Close,Volume,VWAP,log_diff_Open,log_diff_Close,...,Volatility_5,Close_Ratio_60,Trend_60,Volatility_60,Close_Ratio_250,Trend_250,Volatility_250,Close_Ratio_1000,Trend_1000,Volatility_1000
0,2014-03-24,18.004499,18.075001,17.430000,17.592501,17.592501,97470000,10.517801,-0.029848,-0.024620,...,0.013803,0.937302,28.0,0.021579,1.091307,132.0,0.017852,1.543695,510.0,0.020746
1,2014-03-25,17.701500,17.948500,17.441999,17.735500,17.735500,88914000,10.524604,-0.016972,0.008096,...,0.013068,0.947010,28.0,0.021531,1.098959,132.0,0.017822,1.554720,511.0,0.020747
2,2014-03-26,17.856501,17.879999,17.170000,17.170500,17.170500,82414000,10.530405,0.008718,-0.032376,...,0.015723,0.919077,28.0,0.021814,1.062936,131.0,0.017946,1.503821,510.0,0.020752
3,2014-03-27,17.157499,17.200001,16.544001,16.923500,16.923500,115328000,10.538204,-0.039932,-0.014490,...,0.015532,0.908080,28.0,0.021835,1.046650,131.0,0.017931,1.480868,510.0,0.020757
4,2014-03-28,17.002501,17.350000,16.804001,16.914499,16.914499,79736000,10.543578,-0.009075,-0.000532,...,0.016680,0.910059,27.0,0.021733,1.045124,130.0,0.017927,1.478804,509.0,0.020712
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2683,2024-11-18,204.149994,204.669998,200.949997,201.699997,201.699997,36512500,64.009242,-0.012704,-0.004502,...,0.025409,1.068257,33.0,0.018643,1.140440,131.0,0.017179,1.361395,511.0,0.022183
2684,2024-11-19,199.330002,205.300003,198.779999,204.610001,204.610001,31197900,64.023812,-0.023893,0.014324,...,0.026118,1.080892,34.0,0.018652,1.155308,132.0,0.017166,1.380619,511.0,0.022187
2685,2024-11-20,202.979996,203.130005,199.449997,202.880005,202.880005,32769000,64.038924,0.018146,-0.008491,...,0.020649,1.068952,34.0,0.018588,1.144088,131.0,0.017139,1.368535,511.0,0.022187
2686,2024-11-21,203.490005,203.490005,195.750000,198.380005,198.380005,58800000,64.065154,0.002509,-0.022430,...,0.021323,1.042717,34.0,0.018755,1.117410,130.0,0.017205,1.337843,510.0,0.022193


In [8]:
df2 = pd.read_csv("stock_news_sentiment_analysis_results_AMZN.csv")
df3 = df2.drop(['Ticker'],axis=1)
df = df3.drop(['weighted positive sentiment score', 'weighted negative sentiment score', 'weighted neutral sentiment score'],axis=1)
df

,Date,weighted compound sentiment score
0,2011-05-02,-0.006200
1,2011-05-12,-0.051600
2,2011-05-23,0.361200
3,2011-06-20,0.493900
4,2011-06-27,0.831831
...,...,...
1974,2024-11-05,0.429963
1975,2024-11-06,0.538083
1976,2024-11-07,0.400784
1977,2024-11-08,0.405811


In [9]:
# Merge df and data on the 'Date' column
# This performs a left join, keeping all rows from df and adding matching rows from data
df = pd.merge(df, data, on='Date', how='left')

# Display the modified df
df

,Date,weighted compound sentiment score,Open,High,Low,Close,Adj Close,Volume,VWAP,log_diff_Open,...,Volatility_5,Close_Ratio_60,Trend_60,Volatility_60,Close_Ratio_250,Trend_250,Volatility_250,Close_Ratio_1000,Trend_1000,Volatility_1000
0,2011-05-02,-0.006200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2011-05-12,-0.051600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2011-05-23,0.361200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2011-06-20,0.493900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2011-06-27,0.831831,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1974,2024-11-05,0.429963,196.039993,199.820007,195.990005,199.500000,199.500000,30564800.0,63.795572,-0.002089,...,0.034980,1.083932,35.0,0.017822,1.142968,131.0,0.016858,1.350786,513.0,0.022098
1975,2024-11-06,0.538083,200.009995,207.550003,199.139999,207.089996,207.089996,72292200.0,63.830028,0.020049,...,0.037257,1.121427,35.0,0.018233,1.184687,132.0,0.017007,1.401696,513.0,0.022128
1976,2024-11-07,0.400784,207.440002,212.250000,207.190002,210.050003,210.050003,52878400.0,63.855740,0.036475,...,0.026541,1.133369,36.0,0.018279,1.199714,133.0,0.017009,1.421216,513.0,0.022133
1977,2024-11-08,0.405811,209.720001,209.960007,207.440002,208.179993,208.179993,36075800.0,63.873053,0.010931,...,0.020244,1.120198,35.0,0.017591,1.187280,132.0,0.016978,1.408074,512.0,0.022134


In [10]:
df = df.dropna()

In [11]:
df

,Date,weighted compound sentiment score,Open,High,Low,Close,Adj Close,Volume,VWAP,log_diff_Open,...,Volatility_5,Close_Ratio_60,Trend_60,Volatility_60,Close_Ratio_250,Trend_250,Volatility_250,Close_Ratio_1000,Trend_1000,Volatility_1000
81,2014-04-02,0.440400,17.299500,17.415001,17.018999,17.098000,17.098000,89510000.0,10.560462,0.023098,...,0.012529,0.926860,28.0,0.021917,1.053238,130.0,0.017913,1.490937,509.0,0.020718
82,2014-04-24,0.728228,16.483500,16.870001,16.147499,16.857500,16.857500,185874000.0,10.664505,-0.010231,...,0.020544,0.961313,30.0,0.023805,1.027297,128.0,0.017752,1.452669,508.0,0.020635
83,2014-05-06,0.128000,15.476500,15.490500,14.852000,14.869000,14.869000,93646000.0,10.715998,0.010261,...,0.022745,0.867451,31.0,0.021844,0.901790,128.0,0.019042,1.273696,510.0,0.020844
84,2014-05-07,-0.709600,14.778000,14.820000,14.334000,14.635500,14.635500,140304000.0,10.721614,-0.046183,...,0.021755,0.856668,31.0,0.021901,0.887278,127.0,0.019067,1.252815,509.0,0.020850
85,2014-05-20,0.051600,14.855000,15.223000,14.837500,15.059500,15.059500,81914000.0,10.749172,0.004520,...,0.014883,0.903452,31.0,0.022072,0.909894,127.0,0.019176,1.280555,511.0,0.020806
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1973,2024-11-04,0.023137,196.449997,197.330002,194.309998,195.779999,195.779999,38492100.0,63.781771,-0.012897,...,0.034657,1.066879,34.0,0.017706,1.123117,131.0,0.016868,1.325993,512.0,0.022092
1974,2024-11-05,0.429963,196.039993,199.820007,195.990005,199.500000,199.500000,30564800.0,63.795572,-0.002089,...,0.034980,1.083932,35.0,0.017822,1.142968,131.0,0.016858,1.350786,513.0,0.022098
1975,2024-11-06,0.538083,200.009995,207.550003,199.139999,207.089996,207.089996,72292200.0,63.830028,0.020049,...,0.037257,1.121427,35.0,0.018233,1.184687,132.0,0.017007,1.401696,513.0,0.022128
1976,2024-11-07,0.400784,207.440002,212.250000,207.190002,210.050003,210.050003,52878400.0,63.855740,0.036475,...,0.026541,1.133369,36.0,0.018279,1.199714,133.0,0.017009,1.421216,513.0,0.022133


In [49]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import numpy as np

# Assuming `df` is your DataFrame
features = ['weighted compound sentiment score','Log_Returns','Close_Ratio_2', 'Trend_2', 'Close_Ratio_5', 'Trend_5','Close_Ratio_60', 'Trend_60', 'Volatility_60',
       'Close_Ratio_250', 'Trend_250', 'Volatility_250', 'Close_Ratio_1000',
       'Trend_1000', 'Volatility_1000']
target = ['Target']
# Split data into training (80%) and testing (20%) sets
split_index = int(len(df) * 0.8)
train_data = df.iloc[:split_index]
test_data = df.iloc[split_index:]

# Separate features (X) and target (y)
X_train = train_data[features]
y_train = train_data[target]
X_test = test_data[features]
y_test = test_data[target]

# Initialize and train the Random Forest model
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)

# Predict probabilities for the test set
y_prob = rf_model.predict_proba(X_test)[:, 1]  # Probability for the positive class

# Custom threshold for classification
custom_threshold = 0.45  # Change this to your desired threshold
y_pred = (y_prob >= custom_threshold).astype(int)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

# Print evaluation results
print(f"Accuracy: {accuracy:.2f}")
print("\nConfusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(class_report)

# Optional: Display feature importances
feature_importances = pd.DataFrame({
    'Feature': features,
    'Importance': rf_model.feature_importances_
}).sort_values(by='Importance', ascending=False)

print("\nFeature Importances:")
print(feature_importances)


C:\Users\yaman\AppData\Local\Temp\ipykernel_9200\383397968.py:24: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf_model.fit(X_train, y_train)


Accuracy: 0.50

Confusion Matrix:
[[ 37 102]
 [ 44 111]]

Classification Report:
              precision    recall  f1-score   support

         0.0       0.46      0.27      0.34       139
         1.0       0.52      0.72      0.60       155

    accuracy                           0.50       294
   macro avg       0.49      0.49      0.47       294
weighted avg       0.49      0.50      0.48       294


Feature Importances:
                              Feature  Importance
6                      Close_Ratio_60    0.089621
14                    Volatility_1000    0.084713
4                       Close_Ratio_5    0.082989
0   weighted compound sentiment score    0.082718
8                       Volatility_60    0.081911
9                     Close_Ratio_250    0.080529
11                     Volatility_250    0.080445
1                         Log_Returns    0.076433
12                   Close_Ratio_1000    0.075751
2                       Close_Ratio_2    0.073020
10                  

In [14]:
y_test

1573    1.0
1574    1.0
1575    1.0
1576    1.0
1577    0.0
       ... 
1973    1.0
1974    1.0
1975    1.0
1976    0.0
1977    0.0
Name: Target, Length: 294, dtype: float64

In [15]:
# Count unique values in y_test
unique_counts = pd.Series(y_test).value_counts()

# Display the counts
print(unique_counts)

Target
1.0    155
0.0    139
Name: count, dtype: int64


In [51]:
features = ['Log_Returns','Close_Ratio_2', 'Trend_2', 'Close_Ratio_5', 'Trend_5','Close_Ratio_60', 'Trend_60', 'Volatility_60',
       'Close_Ratio_250', 'Trend_250', 'Volatility_250', 'Close_Ratio_1000',
       'Trend_1000', 'Volatility_1000']
target = ['Target']


import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import numpy as np

# Assuming `df` is your DataFrame
features = features
target = 'Target'

# Split data into training (80%) and testing (20%) sets
split_index = int(len(df) * 0.8)
train_data = df.iloc[:split_index]
test_data = df.iloc[split_index:]

# Separate features (X) and target (y)
X_train = train_data[features]
y_train = train_data[target]
X_test = test_data[features]
y_test = test_data[target]

# Initialize and train the Random Forest model
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)

# Predict probabilities for the test set
y_prob = rf_model.predict_proba(X_test)[:, 1]  # Probability for the positive class

# Custom threshold for classification
custom_threshold = 0.45  # Change this to your desired threshold
y_pred = (y_prob >= custom_threshold).astype(int)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

# Print evaluation results
print(f"Accuracy: {accuracy:.2f}")
print("\nConfusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(class_report)

# Optional: Display feature importances
feature_importances = pd.DataFrame({
    'Feature': features,
    'Importance': rf_model.feature_importances_
}).sort_values(by='Importance', ascending=False)

print("\nFeature Importances:")
print(feature_importances)


Accuracy: 0.48

Confusion Matrix:
[[44 95]
 [57 98]]

Classification Report:
              precision    recall  f1-score   support

         0.0       0.44      0.32      0.37       139
         1.0       0.51      0.63      0.56       155

    accuracy                           0.48       294
   macro avg       0.47      0.47      0.46       294
weighted avg       0.47      0.48      0.47       294


Feature Importances:
             Feature  Importance
5     Close_Ratio_60    0.095151
13   Volatility_1000    0.093358
3      Close_Ratio_5    0.091932
0        Log_Returns    0.088390
10    Volatility_250    0.087756
7      Volatility_60    0.087396
11  Close_Ratio_1000    0.085712
8    Close_Ratio_250    0.083816
1      Close_Ratio_2    0.083108
12        Trend_1000    0.055313
9          Trend_250    0.053111
6           Trend_60    0.045689
4            Trend_5    0.032293
2            Trend_2    0.016975
